 ## Hotel Booking Demand. EDA + Predictive Model + Step to Step Data Science Project. 

![](https://img.nh-hotels.net/nh_alicante-069-hotel_facilities.jpg)


* > #### Is there any other way to render tables in a jupyter notebook with quantitative analysis results in a more beautiful fashion?. 

* > #### Data Processing : Does Splitting Data into multiple dataframes, each containing data of a particular type (discrete, continuous, nominal, ordinal, interval, interval ratio, etc. ) help?.


* > #### See how to programmatically give answers to questions like : 
    * Do customer type influence the number of cancelations?
    * Do desposit type shows a higher cancelation rate?.
    * Do customer of a hotel cancel more than other customer form other hotels?.
    * Do we see booking changes occuring at a particular time slot throughout the year,do happen more changes in the summer, what other than time patterns can be found?.
    * Which subset of days since a customer is in the waiting list shows a higher probability of cancelation?. 
    * Which market_segment does better(lower percentage cancelation). 
    * Whats the distribution of children, adults, required_car_parking_spaces and other discrete data. 
        
        
* > #### Amongst other methods for feature selection the estimation of how strong the relationships between the variables are stands out as a useful way of indiciating where there might a strong dependancy between variables. 








*** 
To give a solid ground from where to answer these and other questions I decided to take on this project with the idea of further improving the data preparation steps, sorting chronologically the steps that are to be followed on similar data science projecs.



> ### Data Ingestion 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        if ".csv" in filename:
            df=pd.read_csv(path)
        
        
# Any results you write to the current directory are saved as output.

> ### Function to Beautify Renderization of HTML Tables in Jupyter Notebooks

In [ ]:

from IPython.display import display_html
def display_side_by_side(*args, is_styler=(True, True), title=None):
    html_str=''
    
    for i, container in enumerate(args): 
        if is_styler[i]:
            html_str+=container.render()
        else: 
            html_str+=container.to_html()
            
            
    if title:
        
        import re 

        pattern = "<thead>(.*)</thead>"
        pattern = re.compile(pattern)
        extracted_block = re.search(pattern, html_str).group(1) 
        new_block =  '<caption>{}</caption> \n'.format(title) + extracted_block
        
        block_text = new_block.split("\n")[0]
        html_str = html_str.replace(extracted_block, new_block)
    
    
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
#### Styling Configuration / 
import random
style_props = [dict(selector="caption",  props=[("text-align", "center"),
                                                  ("font-size", "110%"),
                                                  ("color", "black"),
                                                  ('text-decoration', 'underline')]),
              
              dict(selector='th', props=[("text-align", "left")]),
              dict(selector="td", props=[("text-align", "left"), ("border", "1px solid black")])]


r = lambda : random.randint(150,256)

get_hex_color = lambda : "#%02X%02X%02X" % ((r(), r(), r()))
get_hex_prop = lambda: "background-color : {}".format(get_color())

In [ ]:

def set_background_colors(series):
    
    color = get_hex_color()
    background_vector = [ "background-color : {}".format(color) if value != ""
                           else "background-color : turquoise" for value in series ]
    
    return background_vector

    
def BeautifiedFrame(df, hide_index=True, callback=True, subset=None):
   
    
    ### We dont want to show the index when rendering the layout on the window. 
    if hide_index:
        view = df.style.hide_index()
    else: 
        view = df.style
    
    ### Apply color background to all the dataframe
    if not callback:
        properties["background-color"] = get_hex_color()
        view = view.set_properties(**properties)

    ### Apply color mapping with certain conditions on column or row wise. 
    if callback:
        view = view.apply(set_background_colors, axis=0, subset=subset)        
    
    
   
    return view

### Data Preparation 

A few aspects must be tackled during Data Preparation : 


    - Categorical, Nummerical, or a mixture of both?.
    - Are there missing values, None, nan values?.
    - If so, whats the best strategy or method to fill those instances. 
    - Do we need to drop a column or columns?. 
    

> ### Exploring Data and its Types

* By splitting data into its multiple subtypes, **Categorical[Nominal, Ordinal] and Nummerical [Discrete, Continuous, Interval, Interval Ratio] **we often facilitate the visualization of the data, because we simply gor from one single original df with > 10 columns to small dataframes with hardly more than 10 columns. Thus, visualization and an early grasp of whats in the data is facilitated.

* This approach also helps to break down data preparation into multiple steps, without missing important ones, like **inputing missing data or finding outliers**, perform univariate analysis, bivariate analysis and multivariate analysis, and find relationships between variables, both, between and within groups. From the probabilities that emerge from these various statistical test we are more prepared to take on modeling and to build a model to predict a number "Regression" or a category "Classification". 


In [ ]:


dfExploration = df.dtypes.reset_index().rename(columns={"index": "columns", 0: "type"}).sort_values(by="type")
viewExploration = dfExploration[dfExploration.columns[::-1]]

### View Break Down of variables based on their data type classes. 
cat_columns = dfExploration.where(dfExploration["type"] == "object")["columns"].dropna().to_list()
int_columns = dfExploration.where(dfExploration["type"] == "int64")["columns"].dropna().to_list()
float_columns = dfExploration.where(dfExploration["type"] == "float")["columns"].dropna().to_list() 

view_variables = pd.DataFrame([int_columns, cat_columns,  float_columns]).T
view_variables.columns = ["\'int64\' Variables","\'object\' Variables",  "\'float\' Variables"]
view_variables = view_variables.fillna("")

### Apply Styling to the View of our DataFrame.
view_variables = BeautifiedFrame(view_variables, callback=True)
view_variables.set_caption("Pandas Infered Data Types ").set_table_styles(style_props)




* > #### The way pandas handles data ingestion is not always as straight as one might think. Particularly pandas own data types, those that might be inferred when reading a csv or xlsx file were at first misleading. So if you are giving your first steps with Data Wrangling in pandas be aware that although pandas dtype("object) is often associated with categorical data, that is not always the case.

* > #### That is the case for columns "arrival_date_month" and "reservation_status_date" where data is stored as object, which turns out to be a string. A first glimpse to the names will direct your future decissions as for which columns to keep as categorical and which not, based on your requirements and the type of analysis you want to carry out.

* > #### Given that time is an important factor to analyze in this projec where we aim to find patterns like trends, periodicty, frequencies, correlations and relationships between variables, amongst others. 

* >  #### Its equally important to keep an eye on the columns that are not treated as "object" in pandas. That is the case for "agent", "company", that have been stored as "float64". Instead, these fall in the group of categorical variables, specifically of the type ordinal.

In [ ]:
def explore_na(df):
    na_columns = df.isna().sum()
    summary_na = (na_columns[na_columns != 0]/df.shape[0]).sort_values(ascending=False)
    summary_na = summary_na.round(1)
    
    if len(summary_na) == 0:
        print("Non NaN values have been found")
    return summary_na.reset_index().rename(columns = {"index": "Variables with ", 0:"% NaN"}).style.hide_index().set_properties(**{"text-align": "left"})
    
df_na = explore_na(df)
df_na

> ### Splitting Data into its multiple types and sub-types: 
    > * Categorical : Nominal, Ordinal, Dichotomous, 
    > * Nummerical : Discrete, Continuous, Ratio,Interval Ratio, 

In [ ]:

columns_discrete = ["required_car_parking_spaces", 
           "stays_in_weekend_nights",
           "stays_in_week_nights",
           "adults",
           "children",
           "babies",
           "previous_cancellations",
           "previous_bookings_not_canceled",
           "booking_changes",
           "total_of_special_requests",
           "days_in_waiting_list"]


columns_ordinal = ["agent", "company"]
columns_dichotomous = ["is_canceled", "is_repeated_guest", "hotel"]
columns_continuous = ["adr", "lead_time"]

columns_nominal =  [ col for col in cat_columns if "date" not in col and col not in columns_dichotomous]
columns_time = [col for col in df.columns.values if "date" in col]


##### DataFrames ######

### Categorical Data
dfOrdinal = df[columns_ordinal]
dfNominal = df[columns_nominal]
dfDichotomous = df[columns_dichotomous]

### Nummerical Data 
dfDiscrete = df[columns_discrete]
dfContinuous = df[columns_continuous]
dfIntervalRatio = df[columns_time]


### View Break Down of variables based on their data type classes. 
ordinal_columns = dfOrdinal.columns.to_list()
nominal_columns = dfNominal.columns.to_list()
dic_columns = dfDichotomous.columns.to_list()
discrete_columns = dfDiscrete.columns.to_list()
continuous_columns = dfContinuous.columns.to_list()
interval_ratio_columns = dfIntervalRatio.columns.to_list()


### Sorting the dataframe on the horizontal axis to fit the descending order : from types with more variables to less. 
columns = [discrete_columns,nominal_columns, interval_ratio_columns, 
           dic_columns, continuous_columns, ordinal_columns]

### Passing all the columns = series to a new dataframe and transposing to get - surely exists a better workaround to get this done. 
alldata = pd.DataFrame(columns).T
allcolumns = ["dfDiscrete", "dfNominal","dfIntervalRatio",
              "dfDichotomous", "dfContinuous","dfOrdinal"]

alldata.columns = allcolumns

### Inputing "" for None Values that are by default by pandas assigned to the cells when joining data. 
alldata = alldata.fillna("")


### Apply Styling to the View of our DataFrame.
view_variables = BeautifiedFrame(alldata)
view_variables.set_caption("Break Down of Data - DataFrames and their Columns ").set_table_styles(style_props)



> ## Exploring Ordinal Data

 ####     As seen in the Kernel the two columns that we find in our dataframe containing ordinal data are described as follows: 
       
       * "company" : "ID of the company/entity that made the booking or responsible for paying the booking. ID is                              presented instead of designation. 
       
       * "agent" : ID of the Travel Agency that made the booking. 

> ### Ranking the best N Travel Agencies and Companies based on Cancelation Rates.  
  
#### Note : An important feature of this section is the leveraging of the Pandas Style Api which provides a fine application of styling to the html rendered in our jupyter notebook. Some of the code is hardcoded, but a good bulk of it its on the way to be a more production ready code to be used to gain insights during the first steps of any kind of exploratio. 

In [ ]:
def GetTopPerformances(df, column_target,
                       renamed_column,
                       filter_column,
                       volume,
                       top_n, 
                       sort_column=(None, None), 
                       th=None, 
                       captions = None):
    
    ## df, agent, Agent, "TotalBookings", 500, 25. 
    
    ################################################
    performance_df = df[[column_target, "is_canceled"]]
    performance_df = performance_df[performance_df[column_target].notna()]

    Performance = performance_df.groupby(performance_df.columns.to_list()).size().reset_index().rename(columns = {0:"Totals"})
    Performance = pd.pivot_table(Performance, columns = ["is_canceled"], index = [column_target]).reset_index()
    new_columns = [renamed_column, "Not Cancelations", "Cancelations"]

    Performance.columns = new_columns 

    Performance["TotalBookings"] = Performance[renamed_column].apply(lambda x : performance_df[performance_df[column_target] == x].shape[0])
    Performance["PctCancelations"] = (Performance["Cancelations"]/Performance["TotalBookings"])*100
    Performance["PctNotCancelations"] = (Performance["Not Cancelations"]/Performance["TotalBookings"])*100
    Performance["PctCancelations"] = Performance["PctCancelations"].round(2)
    Performance["PctNotCancelations"] = Performance["PctNotCancelations"].round(2)


    AllPerformance = Performance[["Cancelations", "PctCancelations", "Not Cancelations", "PctNotCancelations",
                                  "TotalBookings", renamed_column]]
    FinalPerformance = AllPerformance[AllPerformance[filter_column] > volume].fillna(0)


    ################################################
    if sort_column is None:
        sort_column = ("PctCancelations", "PctNotCancelations")
        
    TopCanc = FinalPerformance[[renamed_column, "Cancelations","TotalBookings","PctCancelations"]]
    TopCanc = TopCanc.sort_values(by=sort_column[0], ascending=False)[:top_n]
    
    TopNoCanc = FinalPerformance[[renamed_column, "Not Cancelations","TotalBookings","PctNotCancelations"]]
    TopNoCanc = TopNoCanc.sort_values(by=sort_column[1], ascending=False)[:top_n]
  
    
    color1 = get_hex_color()
    color2 = get_hex_color()

    stylerCancelations = BeautifiedFrame(TopCanc,
                                         callback=False,
                                         subset=["PctCancelations",
                                         renamed_column])
    
    if captions: 
        stylerCancelations = stylerCancelations.set_caption(captions[0]).set_table_styles(style_props)

    stylerNoCancelations = BeautifiedFrame(TopNoCanc,
                                           callback=False,
                                           subset=["PctNotCancelations",
                                           renamed_column])
    if captions: 
        stylerNoCancelations = stylerNoCancelations.set_caption(captions[1]).set_table_styles(style_props)
    
    
    def highlight_over(row, threshold): 
        
        if row[3] < threshold:
            
            return  [ "background-color: yellow",
                       "background-color: white",
                       "background-color: white",
                       "background-color: yellow" ]
                     
        else: 
            
            return  [ "background-color: {}".format(color1), 
                      "background-color: white",
                      "background-color: white",
                      "background-color: {}".format(color2)]
        
    
    def highlight_under(row, threshold): 
        
        if row[3] > threshold:
            return  [ "background-color: yellow",
                       "background-color: white",
                       "background-color: white",
                       "background-color: yellow" ]
                     
        else: 
            return  [ "background-color: {}".format(color1), 
                      "background-color: white",
                      "background-color: white",
                      "background-color: {}".format(color2)]
        
        
    

    if th:
        stylerCancelations = stylerCancelations.apply(highlight_over,threshold=100-th, axis=1)  
        stylerNoCancelations = stylerNoCancelations.apply(highlight_under,threshold=th, axis=1)
    
    
    return stylerCancelations, stylerNoCancelations



In [ ]:
dfOrdinal.head()

n = 25 ## top N per Volume
df1 = dfOrdinal["agent"].where(dfOrdinal["agent"].notna()).value_counts().reset_index()
df1.columns = ["Agent", " TotalBookings"]
df1 = df1[df1.columns[-1::-1]]

df2 = dfOrdinal["company"].where(dfOrdinal["company"].notna()).value_counts().reset_index()
df2.columns = ["Company", "TotalBookings"]
df2 = df2[df2.columns[-1::-1]]


properties = {"text-align": "center",
              "border-color": "black", 
              "border":"1px solid black"}

TopVolumeAgents = df1[:n]
TopVolumeCompanies = df2[:n]

Stylerdf1 = BeautifiedFrame(TopVolumeAgents[:], callback=True, subset="Agent").set_caption("Top 25 Agencies [Id] Volume").set_properties(**properties)
Stylerdf2 = BeautifiedFrame(TopVolumeCompanies[:], callback=True, subset="Company").set_caption("Top 25 Companies [Id] Volume").set_properties(**properties)

Stylerdf1 = Stylerdf1.set_table_styles(style_props)
Stylerdf2 = Stylerdf2.set_table_styles(style_props)

display_side_by_side(Stylerdf1, Stylerdf2, is_styler=(True,True))


> ### Viewing Top Travel Agencies per Volume(Total Bookings) and % Cancelation

    - Highlighted in yellow those Travel Agencies with a better ratio of Not Cancelations (lowest cancelation rates)

In [ ]:
### Performance here is understood, as agents companies or entities with a low percentage of cancelations.
### Additional information of fees, contracts, payments associated with those companies or agents, etc are out of the scope
### of this rather rough ranking of the best "agents", "companies", etc. 

### The aim with this notebook was to explore in deep the functionalities that emerge from utilizing panda's styling API. 
### In the process of creating more actionable data is where can see where and when steps in EDA or Data Science Projects could
### be pulled off so optimization of time and resources are leveraged at best. 

### In the moment highlighting of background in rows, columns, cells are conditioned to the input parameters 
### that are passed to GetTopPerformances as well as for the logic that can be seen in

target_column = "agent" ### Column to estimate the performance with. 
top_n = 25  ### Number of Top Agents, Companies, etc to be shown in the table. 



caption1 = "Top {} Agencies [Id] based on % Cancelations".format(top_n)  
caption2 = "Top {} Agencies [Id]  based on % Not Cancelations".format(top_n)  
    
styler1, styler2 = GetTopPerformances(df, column_target=target_column,
                                          renamed_column = target_column.capitalize(),
                                          filter_column="TotalBookings", 
                                          volume=100,
                                          top_n=25, 
                                          sort_column=("TotalBookings", "TotalBookings"), 
                                          th = 80, captions=[caption1, caption2])


target_column = "company"
caption3 = "Top {} Companies [Id] based on \n % Cancelations".format(top_n)  
caption4 = "Top {} Companies [Id] based on % Cancelations".format(top_n) 

styler3, styler4 = GetTopPerformances(df, column_target=target_column,
                                          renamed_column = target_column.capitalize(),
                                          filter_column="TotalBookings", 
                                          volume=100,
                                          top_n=25, 
                                          sort_column=("TotalBookings", "TotalBookings"),
                                          th=90, captions=[caption3, caption4])
    
display_side_by_side(styler1, styler2, styler3, styler4, is_styler=(True, True, True, True))

 * > #### We focused on looking at how well or bad agencies and companies perform, by calculating cancelation rates we obtain a good marker of how they are doing and we can drive our business decissions to improve the rate of no cancelations by inspecting at how worth is to invest at a particular entity. 



In [ ]:
reversed_columns = ordinal_columns[-1::-1]
subset_orig = df[reversed_columns + ["is_canceled"]]
subset_orig = subset_orig.fillna(-1)

condition = (subset_orig.is_canceled == 1) & (subset_orig.agent != -1) & (subset_orig.company != -1)
aggregated_df = subset_orig.where(condition).groupby(reversed_columns).size().reset_index().rename(columns = {0: "Total"})
aggregated_df = aggregated_df.sort_values(by="Total", ascending=False)
aggregated_df = aggregated_df.rename(columns = {"Total": "Cancelations"})[:10]

caption1 = "Count Travel Agencies - Companies Pair instances based on Total Cancelations"  
Cancelations = BeautifiedFrame(aggregated_df, callback=True).set_caption(caption1).set_table_styles(style_props)



condition = (subset_orig.is_canceled == 0) & (subset_orig.agent != -1) & (subset_orig.company != -1)
aggregated_df = subset_orig.where(condition).groupby(reversed_columns).size().reset_index().rename(columns = {0: "Total"})
aggregated_df = aggregated_df.sort_values(by="Total", ascending=False)
aggregated_df = aggregated_df.rename(columns = {"Total": "Not Cancelations"})[:10]

caption2 = "Count Travel Agencies - Companies Pair instances based on Total Bookings(Not Cancelations)"

NotCancelations = BeautifiedFrame(aggregated_df, callback=True).set_caption(caption2).set_table_styles(style_props)
    
display_side_by_side(Cancelations, NotCancelations, is_styler=(True, True))

#print("The total number of cancelations where agent and company is :  ", )

   >* In a similar fashion than the previous computation we now look at unique pairs of agencies and companies, to see if there is any pair that shows a high or low cancelation.  

> ### Exploring Nummerical Variables


> #### Continuous Variables : These show

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns

sns.set()

fig = plt.figure(figsize=(25,10))

ax1 = fig.add_subplot(2, 1, 1)
sns.distplot(dfContinuous["adr"],ax=ax1)

ax2 = fig.add_subplot(2, 1, 2)
sns.distplot(dfContinuous["lead_time"],ax=ax2)

plt.show()


- Computing Probability of No Cancelations based on total days_in_waiting_list. 
- How likely is a customer to not cancel in a given period of time?.
    
    Time Periods are broken down as follows: 
    
    **"3 Month Probs"**
       - Probability of No Cancelation in the first 3 months : Interval [0,90] (days)
       - Probability "  "  "   between month 3 and month 6: Interval [90,180] 
    
    **"2 Month Probs"**
       - Probability of No Cancelation in the first 2 months : Interval [0,60] (days)
       - Probability "  "  "   between month 2 and month 4: Interval [60,120] (days)
     
    **"1 Month Probs"**
       - Probability of No  Cancelation in the first 2 months : Interval [0,60] (days)
       - Probability "  "  "   between month 2 and month 4: Interval [60,120] (days)
     
    **"15 Days Probs"**
   
       - Probability of No Cancelation in the first 15 days: Interval [0,15] (days)
       - Probability of No Cancelation between day 15 and day 30: Interval [15,30] (days)
    
    **"Weekly Probs"**
       - Probability of No Cancelation in the first 7 days: Interval [0,7] (days)
       - Probability of No Cancelation between day 15 and day 30: Interval [7,15] (days)
   
    **"3Days Probs"**
       - Probability of No Cancelation in the first 3 days: Interval [0,3] (days)
       - Probability of No Cancelation between day 3 and day 6: Interval [3,6] (days)
   
    
    

In [ ]:
def get_df(df, condition_value):
    
    df_selection = df[["days_in_waiting_list", "is_canceled"]]
    condition = df_selection.is_canceled == condition_value
    
    if condition_value == 0: 
        new_column_name = "NoCancelations"
    else: 
        new_column_name = "Cancelations"

    df_selection = df_selection.where(condition)["days_in_waiting_list"].value_counts().to_frame().reset_index()
    df_selection.columns = ["days_in_waiting_list", new_column_name]

    df_selection = df_selection[df_selection["days_in_waiting_list"] != 0].sort_values(by="days_in_waiting_list", ascending=True)
    df_selection = df_selection.fillna(0)
    df_selection["days_in_waiting_list"].fillna(0, inplace=True)
    return df_selection


In [ ]:
def find_best_time(orig_df, step, is_cancelation=False):
    
    step = step
    min_x = 0 
    max_x = step
    max_threshold = orig_df["days_in_waiting_list"].max()
    
    mapping = {}
    interval_data = {}
    it = 1 
    

    if is_cancelation:
        columns =  ["Cancelations", "ProbCancelations"]
    else: 
        columns = ["NoCancelations", "ProbNoCancelations"]
    
    while(max_x < max_threshold): 
        
        condition = (orig_df["days_in_waiting_list"] >= min_x) & (orig_df["days_in_waiting_list"] <= max_x)
        values  = orig_df[condition][columns[0]]
        if len(values) == 0:
            rolled_mean = 0
        else: 
            rolled_mean = values.mean()
        
        mapping[it] = "[{}-{}]".format(min_x, max_x)
        interval_data[it] = rolled_mean
        min_x += step
        max_x += step
        it+=1

    total_cancelations = sum(interval_data.values())
    probs = [ value/total_cancelations*100 for value in list(interval_data.values())]
    
    
    ### Composing the new dataframe with the probabilities and the Interval-Ratio Data. 
    final_df = pd.DataFrame(probs, index=interval_data.keys())
    final_df["Interval"] =  [ mapping[value] for value in final_df.index ]
    
    final_df.fillna(0, inplace=True)
    
    final_df.columns = [columns[1], "Interval"]
    final_df = final_df[["Interval", columns[1]]]
    final_df.sort_values(by=columns[1], ascending=False, inplace=True)
   
    return final_df

In [ ]:

def show_best_days(orig_df, is_cancelation=False):

    periods = [1,2,7,15,30,60,90]
    stylers = []
    captions = ["2Days Probs", "3Days Probs", "Weekly Probs",
                "15 Days Probs", "Month Probs", "2 Month Probs", "3 Month Probs"]

    for i, period in enumerate(periods): 
        best_times = find_best_time(orig_df, period, is_cancelation)
        styler = BeautifiedFrame(best_times, hide_index=False).set_table_styles(style_props).set_caption(captions[i])
        stylers.append(styler)


    display_side_by_side(stylers[6], stylers[5], stylers[4],  is_styler=(True, True, True))
    display_side_by_side(stylers[3], stylers[2], stylers[1], is_styler=(True, True, True))

In [ ]:
WaitingListNoCancelation = get_df(df, 0)
WaitingListCancelation = get_df(df, 1)



In [ ]:
show_best_days(WaitingListNoCancelation, is_cancelation= False)


* We see the probabilities equally distributed across time periods. Thus, we can say there is no special time of the year when customers do keep their booking status unchanged.

* Computing Probability of Cancelations based on total days_in_waiting_list. 
    
    **"3 Month Probs"**
       - Probability of Cancelation in the first 3 months : Interval [0,90] (days)
       - Probability "  "  "   between month 3 and month 6: Interval [90,180] 
    
    **"2 Month Probs"**
       - Probability of Cancelation in the first 2 months : Interval [0,60] (days)
       - Probability "  "  "   between month 2 and month 4: Interval [60,120] (days)
     
    **"1 Month Probs"**
       - Probability of Cancelation in the first 2 months : Interval [0,60] (days)
       - Probability "  "  "   between month 2 and month 4: Interval [60,120] (days)
     
    **"15 Days Probs"**
   
       - Probability of Cancelation in the first 15 days: Interval [0,15] (days)
       - Probability of Cancelation between day 15 and day 30: Interval [15,30] (days)
    
    **"Weekly Probs"**
       - Probability of Cancelation in the first 7 days: Interval [0,7] (days)
       - Probability of Cancelation between day 15 and day 30: Interval [7,15] (days)
   
    **"3Days Probs"**
       - Probability of Cancelation in the first 3 days: Interval [0,3] (days)
       - Probability of Cancelation between day 3 and day 6: Interval [3,6] (days)
   
    
    

In [ ]:
show_best_days(WaitingListCancelation, is_cancelation= True)


* As expected cancelation with respect of time period show a different pattern, that is, we can say as we see in the tables that customers cancel more frequently in the following cases : 

    * The first three months - **P=0.4979**.
    * The first two months - **P=0.4012**.
    * The second month of the first two months - **P=0.2721**.
    * Between days 30-45 - **P=0.20**.
    * Betwen days 35-42 - **P=0.1345**. 
    * Between days 38-40 - **P=0.59958**.
    

* These probabilities tell us how certain is that cancelations will occur at certain times while a customer is in the waiting list. 

In [ ]:

data = dfDiscrete["stays_in_weekend_nights"].value_counts()
new_indices = data.index.to_list()
new_indices.pop(0)
#data = data.loc[data.index.to_list().remove(0)]
#data


> ### Visualization - Discrete and Continous Data.

In [ ]:
import matplotlib.pyplot as plt, seaborn as sns
fig = plt.figure(figsize=(25,10))

ax = fig.add_subplot(2,1,1)
data = dfDiscrete["stays_in_weekend_nights"].value_counts()

new_indices = data.index.to_list()
new_indices.pop(0)

data = data.loc[new_indices]

x = data.index.values
y = data.values
sns.barplot(x,y)
ax.set_title("Stays in Weekend Nights")

ax = fig.add_subplot(2,1,2)
data = dfDiscrete["stays_in_week_nights"].value_counts()


new_indices = data.index.to_list()
new_indices.pop(0)
data = data.loc[new_indices]

x = data.index.values
y = data.values
sns.barplot(x,y)
ax.set_title("Stays in Week Nights")
    
plt.show()

In [ ]:
fig = plt.figure(figsize=(25,20))

####### Pie Plot for Adults

fig.add_subplot(4,2,1)
total_count = dfDiscrete["adults"].count()
orig_data = dfDiscrete["adults"].value_counts()

condition_small = (orig_data/total_count*100 < 3)
condition_big = (orig_data/total_count*100 > 3)

data = orig_data[condition_big]
data["others"] = orig_data[condition_small].sum()

labels = data.index.values
totals = data.values

explode_values = [ 0.10 for i  in range(1, len(totals)+1) ]
explode_values.reverse()
plt.pie(totals, labels = labels, explode=explode_values,  autopct='%1.1f%%', shadow=True, startangle=180 )
plt.title("adults", fontsize=25)



####### Pie Plot for children

fig.add_subplot(4,2,2)
total_count = dfDiscrete["children"].count()
orig_data = dfDiscrete["children"].value_counts()

condition_small = (orig_data/total_count*100 < 3)
condition_big = (orig_data/total_count*100 > 3)

data = orig_data[condition_big]
data["others"] = orig_data[condition_small].sum()

labels = data.index.values
totals = data.values

explode_values = [ 0.10 for i  in range(1, len(totals)+1) ]
explode_values.reverse()
plt.pie(totals, labels = labels, explode=explode_values,  autopct='%1.1f%%', shadow=True, startangle=180 )
plt.title("Children", fontsize=25)

####### Pie Plot for Babies

fig.add_subplot(4,2,3)
total_count = dfDiscrete["babies"].count()
orig_data = dfDiscrete["babies"].value_counts()

condition_small = (orig_data/total_count*100 < 3)
condition_big = (orig_data/total_count*100 > 3)

data = orig_data[condition_big]
data["others"] = orig_data[condition_small].sum()


labels = data.index.values
totals = data.values

explode_values = [ 0.3 for i  in range(1, len(totals)+1) ]
explode_values.reverse()
plt.pie(totals, labels = labels, explode=explode_values,  autopct='%1.1f%%', shadow=True, startangle=180 )
plt.title("Babies", fontsize=25)

####### Pie Plot for required Parking Spaces

fig.add_subplot(4,2,4)
total_count = dfDiscrete["required_car_parking_spaces"].count()
orig_data = dfDiscrete["required_car_parking_spaces"].value_counts()

condition_small = (orig_data/total_count*100 < 3)
condition_big = (orig_data/total_count*100 > 3)

data = orig_data[condition_big]
data["others"] = orig_data[condition_small].sum()


labels = data.index.values
totals = data.values

explode_values = [ 0.3 for i  in range(1, len(totals)+1) ]
explode_values.reverse()
plt.pie(totals, labels = labels, explode=explode_values,  autopct='%1.1f%%', shadow=True, startangle=180 )
plt.title("Required Car Parking Spaces", fontsize=25)



####### Pie Plot for required Parking Spaces

fig.add_subplot(4,2,5)
total_count = dfDiscrete["previous_bookings_not_canceled"].count()
orig_data = dfDiscrete["previous_bookings_not_canceled"].value_counts()

condition_small = (orig_data/total_count*100 < 3)
condition_big = (orig_data/total_count*100 > 3)

data = orig_data[condition_big]
data["others"] = orig_data[condition_small].sum()


labels = data.index.values
totals = data.values

explode_values = [ 0.5 for i  in range(1, len(totals)+1) ]
explode_values.reverse()
plt.pie(totals, labels = labels, explode=explode_values,  autopct='%1.1f%%', shadow=True, startangle=180 )
plt.title("Previous Bookings Not Canceled", fontsize=25)



####### Pie Plot for required Parking Spaces

fig.add_subplot(4,2,6)
total_count = dfDiscrete["total_of_special_requests"].count()
orig_data = dfDiscrete["total_of_special_requests"].value_counts()

condition_small = (orig_data/total_count*100 < 3)
condition_big = (orig_data/total_count*100 > 3)

data = orig_data[condition_big]
data["others"] = orig_data[condition_small].sum()


labels = data.index.values
totals = data.values

explode_values = [ 0.15 for i  in range(1, len(totals)+1) ]
explode_values.reverse()
plt.pie(totals, labels = labels, explode=explode_values,  autopct='%1.1f%%', shadow=True, startangle=180 )
plt.title("Total of Special Requests", fontsize=25)



####### Pie Plot for previous cancelations

fig.add_subplot(4,2,7)
total_count = dfDiscrete["previous_cancellations"].count()
orig_data = dfDiscrete["previous_cancellations"].value_counts()

condition_small = (orig_data/total_count*100 < 3)
condition_big = (orig_data/total_count*100 > 3)

data = orig_data[condition_big]
data["others"] = orig_data[condition_small].sum()


labels = data.index.values
totals = data.values

explode_values = [ 0.5 for i  in range(1, len(totals)+1) ]
explode_values.reverse()
plt.pie(totals, labels = labels, explode=explode_values,  autopct='%1.1f%%', shadow=True, startangle=180 )
plt.title("Previous Cancelations", fontsize=25)








## Measure Strongness of Relationship between Continuous and Discrete Data and Dichotomous Variables. 

    
    "is_canceled", "is_repeated_guest", "hotel". 


> "The Point-Biserial Correlation Coefficient is a correlation measure of the strength of association between a continuous-level variable (ratio or interval data) and a binary variable. Binary variables are variables of nominal scale with only two values."

In [ ]:

from scipy.stats import pointbiserialr
def CorrDichNum(dfDichotomous, dfDiscrete, dfContinuous,  
                dic_columns, discrete_columns, continuous_columns):
    
    corr = {}

    for dic_column in dic_columns: 

        
        corr[dic_column] = {}
        mapping = { v:i for i,v in enumerate(dfDichotomous[dic_column].unique()) }

        if "object" not in dfDichotomous[dic_column].dtypes.name :
            dic_array = np.array(dfDichotomous[dic_column].values)
            
        else:
            dic_array = [ mapping[value] for value in dfDichotomous[dic_column]  ]


        for column_disc in discrete_columns:   
            pbc = pointbiserialr(dfDiscrete[column_disc].fillna(method="ffill"), dic_array)
            if pbc.pvalue < 0.05:
                corr[dic_column][column_disc] = abs(pbc.correlation)

        for column_cont in continuous_columns:   
                pbc = pointbiserialr(dfContinuous[column_cont].fillna(method="ffill"), dic_array)
                if pbc.pvalue < 0.05:
                    corr[dic_column][column_cont] = abs(pbc.correlation)


    import operator
    for key in corr.keys():
        corr[key] = dict( sorted(corr[key].items(), key=operator.itemgetter(1),reverse=True) )
                    
                         
    return corr

                     
    
dic_columns = dfDichotomous.columns.values
discrete_columns = dfDiscrete.columns.values
continuous_columns = dfContinuous.columns.values

corr = CorrDichNum(dfDichotomous, dfDiscrete, dfContinuous,
                   dic_columns, discrete_columns, continuous_columns)

##### Correlation between dichotomous variables and countinous or discrete variable(nummerical) 
corr_canceled = pd.DataFrame.from_dict(corr["is_canceled"], orient="index").reset_index().rename(columns={"index": "variable_name",
                                                                                               0: "correlation_value"})

corr_hotel = pd.DataFrame.from_dict(corr["hotel"], orient="index").reset_index().rename(columns={"index": "variable_name",
                                                                                               0: "correlation_value"})

corr_repeated_guest = pd.DataFrame.from_dict(corr["is_repeated_guest"], orient="index").reset_index().rename(columns={"index": "variable_name",
                                                                                               0: "correlation_value"})

##### Tidying columns and editing style. 

corr_canceled = corr_canceled.round(3)
corr_hotel = corr_hotel.round(3)
corr_repeated_guest = corr_repeated_guest.round(3)

corr_canceled.columns = [" \"is_canceled\" correlations", ""]
corr_hotel.columns = [" \"hotel\" correlations", ""]
corr_repeated_guest.columns = [" \"is_repeated_guest\" correlations", ""]






In [ ]:
ViewCorrCanceled = BeautifiedFrame(corr_canceled, callback=True)
ViewCorrHotel = BeautifiedFrame(corr_hotel, callback=True)
ViewCorrRepeatedGuest = BeautifiedFrame(corr_repeated_guest, callback=True)

display_side_by_side(ViewCorrCanceled, ViewCorrHotel, ViewCorrRepeatedGuest, is_styler=(True, True, True))


> ### Exploring Interval Ratio (Time) Variables

In [ ]:
import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

In [ ]:
from datetime import datetime
from time import strptime


columns = ["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"]
AuxDf = dfIntervalRatio.rename(columns={"arrival_date_year": "year",
                                        "arrival_date_month": "month", 
                                        "arrival_date_day_of_month": "day"})

AuxDf["month"] = AuxDf["month"].apply(lambda x : strptime(x, '%B').tm_mon ) 
dfIntervalRatio["Arrival"] = pd.to_datetime(AuxDf[["year", "month", "day"]])


In [ ]:
import numpy as np
dfIntervalRatio["reservation_status_date"] = pd.to_datetime(dfIntervalRatio["reservation_status_date"],
                              format='%Y-%m-%d')

diff = np.mean([datetime.days for datetime in (dfIntervalRatio["reservation_status_date"] - dfIntervalRatio["Arrival"])])

In [ ]:
dfIntervalRatio.columns.values

In [ ]:
months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October",
         "November", "December"]


CancellationsByMonth = df.groupby(["arrival_date_month"])["is_canceled"].count().reset_index()
CancellationsByMonth["arrival_date_month"] = pd.Categorical(CancellationsByMonth["arrival_date_month"], 
                                                            categories = months,
                                                            ordered=True)


n_cancelations = df[df["is_canceled"] == 1].count()[0]
CancellationsByMonth["Total_Cancelations"] = [n_cancelations]*CancellationsByMonth.shape[0]
CancellationsByMonth["Pct_cancelations"] = CancellationsByMonth["is_canceled"]/CancellationsByMonth["Total_Cancelations"]*100


df_month = CancellationsByMonth.sort_values(by="Pct_cancelations", ascending=False)
df_month["Pct_cancelations"] = df_month["Pct_cancelations"].round(2).astype(str) + " %"


In [ ]:

CancellationsByDay = df.groupby(["arrival_date_day_of_month"])["is_canceled"].count().reset_index()


n_cancelations = df[df["is_canceled"] == 1].count()[0]
CancellationsByDay["Total_Cancelations"] = [n_cancelations]*CancellationsByDay.shape[0]
CancellationsByDay["Pct_cancelations"] = CancellationsByDay["is_canceled"]/CancellationsByDay["Total_Cancelations"]*100


df_day = CancellationsByDay.sort_values(by="Pct_cancelations", ascending=False)
df_day["Pct_cancelations"] = df_day["Pct_cancelations"].round(2).astype(str) + " %"


In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(25,10)})

sorted_days = CancellationsByDay.sort_values(by="arrival_date_day_of_month")
plt.plot(sorted_days["arrival_date_day_of_month"],sorted_days["is_canceled"],linewidth=3, color="orange")


plt.title("Cancelations By Day between 2015-2017", fontsize=25)
plt.xlabel("Day", fontsize=20)
plt.ylabel("Total Number of Cancelations", fontsize=20)
plt.xticks(list(range(1,32)))




In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(25,10)})

sorted_months = CancellationsByMonth.sort_values(by="arrival_date_month").set_index("arrival_date_month")

sorted_months["is_canceled"].plot(linewidth=3, color="blue")

plt.title("Cancelations By Month between 2015-2017", fontsize=25, pad=20)
plt.xlabel("Month", fontsize=20)
plt.ylabel("Cancelations", fontsize=20)


> * We don't see a particular pattern in the time data other than there seems to be a higher number of cancelations in the summer period. 


> ### Stats that lead to Feature Selection

* > ####  Its now that the previous fragmentation of the dataset into various dataframes makes sense. 
* > #### Our goal here is to measure the relationship between a combination of variables, their types will define what type of test will help us to come out with certain probabilities that a given relationship exists between these variables.   
* > #### In general we could divide the type of relationships in Categorical to Categorical, Categorical to Nummerical, Nummerical to Nummerical.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import scipy.stats as ss
from statsmodels.multivariate.manova import MANOVA



In [ ]:
def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))



manova = MANOVA(dfDiscrete.fillna(method="ffill"), dfDichotomous["is_canceled"])
print(manova.mv_test())


In [ ]:
def categorical_relationship(cat_x, cat_y):

    #### Split data target value = y and non_target values = X
    y = cat_y.values
    dfdummies = pd.get_dummies(cat_x)

    x = np.array(dfdummies.values)
    chi2Values, pvalues = chi2(x, y)

    for i, chi2pvalue in enumerate(zip(chi2Values, pvalues)):
        if chi2pvalue[1]  < 0.05:
            print("predictor [{}]".format(dfdummies.columns.values[i]))
            print("Chi Value: ", chi2pvalue[0])
            print("p value: ", chi2pvalue[1])

In [ ]:
cat_dict = {}    
for column in dfNominal.columns.values:
        print("################################################################")
        print("Estimating Relationship between [{}] and Target Variable [is_canceled]".format(column))
        categorical_relationship(dfNominal[column], dfDichotomous["is_canceled"])

In [ ]:


def label_encoder(data):

    from sklearn.preprocessing import LabelEncoder
    label_encoder = LabelEncoder()
    data = label_encoder.fit_transform(data)
    classes = label_encoder.classes_
    mappings = { i : classes[i] for i in range(0, len(classes))}
    return data, mappings
    

def onehot_encoder(data, list_features=None):
    
    
    data = np.array(data).reshape(1,)
    print("data after reshaped: ", data)
    from sklearn.preprocessing import OneHotEncoder    
   
    if list_features is None or len(list_features) is 0:
        onehot_encoder = OneHotEncoder(categories="auto")
    else:
        onehot_encoder = OneHotEncoder(categorical_features=list_features)
   

    data = onehot_encoder.fit_transform(data).toarray()
    data = np.delete(data, 0, 1) #Remove one variable to avoid dummy variable trap.
    return data


In [ ]:



def heatmap(self, corr):

    import seaborn as sns
    sns.heatmap(corr,
                xticklabels=corr.columns.values, 
                yticklabels=corr.columns.values, 
                annot=True)


def most_highly_correlated(self, corr_mat, numtoreport):

    try:
        # find the correlations
        cormatrix = corr_mat
        # set the correlations on the diagonal or lower triangle to zero,
        # so they will not be reported as the highest ones:
        cormatrix *= np.tri(*cormatrix.values.shape, k=-1).T
        # find the top n correlations
        cormatrix = cormatrix.stack()
        cormatrix = cormatrix.reindex(cormatrix.abs().sort_values(ascending=False).index).reset_index()
        # assign human-friendly names
        cormatrix.columns = ["FirstVariable", "SecondVariable", "Correlation"]

        report = cormatrix.head(numtoreport)
        self.results.append(report)
        return report, cormatrix

    except Exception as Ex:
        return None

    

> ### More Visualizations - Grid Visuals to Spot Linear Non Linear Relationships between a selection of candidate variables. 

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(4,2, figsize=(15,10))


from random import randint
r = lambda: randint(0,256)
hex_color = lambda: "#%02X%02X%02X" % (r(), r(), r())

    
    
df1 = df[["lead_time", "adr", "is_canceled"]]
df2 = df[["adr", "previous_bookings_not_canceled", "is_canceled"]]
df3 = df[["lead_time", "previous_bookings_not_canceled", "is_canceled"]]
df4 = df[["arrival_date_month", "previous_bookings_not_canceled", "is_canceled"]]
df4 = df4.groupby(["arrival_date_month", "previous_bookings_not_canceled"])["is_canceled"].sum().reset_index()
levels = list(df["is_canceled"].unique())

dfs = [df1, df2, df3, df4]
level_title =  { 0 : "Not Cancelation",
                 1 :  "Cancelation"}

for i, dfplot in enumerate(dfs):
    for j, level in enumerate(levels):
       
        x_column = dfplot.columns.to_list()[0]
        y_column = dfplot.columns.to_list()[1]
        
        df_filtered = dfplot.loc[dfplot.is_canceled == level, :]

        if i>0:
            max_value = max(df_filtered[y_column])
            ylim = (0, int(max_value*1.2))
        else:
            ylim = (0,1000)

        ax[i][j].scatter(x_column, y_column, data=df_filtered, s=4, c=hex_color(), label=str(level) )
        ax[i][j].set_title("{} - {} / {} ".format(level_title[level], x_column, y_column), fontsize=15)
        ax[i][j].set(ylim=ylim)
        if i == 3:
            ax[i][j].set_xticklabels(df_filtered[x_column].unique(), **{"rotation":-65})
        
        
fig.tight_layout(pad=2.5)
    


In [ ]:
def highlight_selection(s, threshold, column, ncolumns):
    value = list(s[column].values[0].split(" %"))[0]
    if float(value) >=threshold[0] and float(value) <=threshold[1] :
        arg = "background-color: turquoise"
    else: 
        arg = ""
    return [arg for i in range(0, ncolumns)]

In [ ]:
def beautify_frame(df, hide_index=True):
    
    if not hide_index:
        return df.style.set_properties(**{"text-align":"left"})
    return df.style.hide_index().set_properties(**{"text-align":"left"})

    

> ## EDA - Many Variables to "is_canceled"

In [ ]:
def pct_aggregator(df, column_target, column_grouping, condition_value):
    
    condition = df[column_target] == condition_value 
    ### Total number of observations where there is a cancelation = bookings where is_canceled ==1
    total_bookings = df.where(condition).count()[1]

    ### Cancelations : Grouping + Aggregation
    group_cancelations = df.where(condition).groupby(column_grouping)[column_target].count()
    cancelations_df = group_cancelations.reset_index().rename(columns={column_target: "TotalCancelations"})
    
    ### Bookings : Grouping + Aggregation
    group_bookings = df.groupby(column_grouping)[column_target].count()
    bookings_df = group_bookings.reset_index().rename(columns={column_target: "TotalBookings"})
    
    result_df = cancelations_df.merge(bookings_df, on = column_grouping[0])
    
    n_cancelations = result_df["TotalCancelations"].sum()
    print("Total cancelations : ", n_cancelations)
    
    ###  Computing the Percentage of Cancelations for each of the groups to which unique MultiIndeces have been generated. 
    result_df["PCT_Cancelations"] = (result_df["TotalCancelations"]/result_df["TotalBookings"])*100
    

    ### Rearranging the columns in the result_df to match the following order: 
    columns = [ column_grouping[0],  "PCT_Cancelations", "TotalCancelations", "TotalBookings"]
    result_df = result_df[columns]

    result_df.sort_values(by="PCT_Cancelations", ascending=False, inplace=True)
    result_df["PCT_Cancelations"] = result_df["PCT_Cancelations"].round(2).astype(str) + " %"
    result_df.sort_values(by=["PCT_Cancelations", "TotalCancelations", "TotalBookings"],
                          ascending=(False, False, False), inplace=True)

    return result_df 

In [ ]:
def pct_multiple_aggregator(df, target_column, group_columns, condition, ascending_order=False): 
    
    clause = df[target_column] == condition
    result_df = df.groupby(group_columns)[target_column].count().reset_index().rename(columns={"is_canceled": "TotalBookings"})
    
    ### Composing new names for the future columns holding pct values and totals. 
    f = lambda: "NoCancelations (Totals)" if condition == 0  else "Cancelations (Totals)"
    name_totals = f()
    name_pct = "Pct" + name_totals

    ### 
    cancelations = df.where(clause).groupby(group_columns)[target_column].count()
    cancelations = cancelations.reset_index().rename(columns={"is_canceled" : name_totals})

    
    result_df = pd.merge(result_df, cancelations, on = group_columns, suffixes=(False, False))
    result_df[name_pct] = result_df[name_totals]/result_df["TotalBookings"]*100
    result_df = result_df[[ group_columns[0], group_columns[1], name_pct, name_totals, "TotalBookings" ]]

    result_df.sort_values(by=name_pct, ascending=ascending_order, inplace=True)
    result_df[name_pct] = result_df[name_pct].round(2).astype(str) + " %"
    return result_df

> ### EDA - Country vs Cancelations

In [ ]:
country_cancelations = pct_aggregator(df, "is_canceled", ["country"], 1)
cancelations1000 = country_cancelations[country_cancelations["TotalBookings"] > 1000]
cancelations100 = country_cancelations[country_cancelations["TotalBookings"] > 100]

caption1 = "Pct Cancelations per Country - Bookings > 100" 
caption2 = "Pct Cancelations per Country - Bookings > 1000" 

ViewCancelations_100 = BeautifiedFrame(cancelations100, callback=True, subset=["country", "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCancelations_1000 = BeautifiedFrame(cancelations1000, callback=True,  subset=["country", "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)

display_side_by_side(ViewCancelations_100, ViewCancelations_1000, is_styler=(True, True))

> ### EDA - Meal vs Cancelations

In [ ]:
meal_cancelations = pct_aggregator(df, "is_canceled",["meal"], 1)
meal_cancelations100 = meal_cancelations[meal_cancelations["TotalBookings"] > 100]
meal_cancelations1000 = meal_cancelations[meal_cancelations["TotalBookings"] > 1000]

caption1 = "Pct Cancelations per Meal - Bookings > 100" 
caption2 = "Pct Cancelations per Meal - Bookings > 1000" 

ViewCancelations_100 = BeautifiedFrame(meal_cancelations100, callback=True, subset=["meal", "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCancelations_1000 = BeautifiedFrame(meal_cancelations1000, callback=True,  subset=["meal", "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)

display_side_by_side(ViewCancelations_100, ViewCancelations_1000, is_styler=(True, True))


> ### EDA - Reserved Room vs Cancelations

In [ ]:
reserved_room_cancelations =  pct_aggregator(df, "is_canceled",["reserved_room_type"], 1)

room_cancelations100 = reserved_room_cancelations[reserved_room_cancelations["TotalBookings"] > 100]
room_cancelations1000 = reserved_room_cancelations[reserved_room_cancelations["TotalBookings"] > 1000]


caption1 = "Pct Cancelations per Reserved Room - Bookings > 100" 
caption2 = "Pct Cancelations per Reserved Room - Bookings > 1000" 


ViewCancelations_100 = BeautifiedFrame(room_cancelations100, callback=True, subset=["meal", "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCancelations_1000 = BeautifiedFrame(room_cancelations1000, callback=True,  subset=["meal", "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)

display_side_by_side(ViewCancelations_100, ViewCancelations_1000, is_styler=(True, True))




> ### EDA - Deposit Type vs Cancelations

In [ ]:
deposit_cancelations = pct_aggregator(df, "is_canceled",["deposit_type"], 1)
deposit_cancelations100 = deposit_cancelations[deposit_cancelations["TotalBookings"] > 100]
deposit_cancelations1000 = deposit_cancelations[deposit_cancelations["TotalBookings"] > 1000]


caption1 = "Pct Cancelations per Deposit Type - Bookings > 100" 
caption2 = "Pct Cancelations per Deposit Type - Bookings > 1000" 


ViewCancelations_100 = BeautifiedFrame(deposit_cancelations100, callback=True, subset=["deposit_type", "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCancelations_1000 = BeautifiedFrame(deposit_cancelations1000, callback=True,  subset=["deposit_type", "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)

display_side_by_side(ViewCancelations_100, ViewCancelations_1000, is_styler=(True, True))




> ### EDA - Customer Type vs Cancelations

In [ ]:
customer_cancelations = pct_aggregator(df, "is_canceled",["customer_type"], 1)


customer_cancelations100 = customer_cancelations[customer_cancelations["TotalBookings"] > 100]
customer_cancelations1000 = customer_cancelations[customer_cancelations["TotalBookings"] > 1000]


caption1 = "Pct Cancelations per Customer Type - Number Bookings > 100" 
caption2 = "Pct Cancelations per Customer Type - Number Bookings > 1000" 


ViewCancelations_100 = BeautifiedFrame(customer_cancelations100, callback=True, subset=["customer_type", "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCancelations_1000 = BeautifiedFrame(customer_cancelations1000, callback=True,  subset=["customer_type", "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)

display_side_by_side(ViewCancelations_1000, ViewCancelations_100, is_styler=(True, True))


> ### EDA - Market Segment vs Cancelations

In [ ]:
segment_cancelations = pct_aggregator(df, "is_canceled",["market_segment"], 1)

segment_cancelations100 = segment_cancelations[segment_cancelations["TotalBookings"] > 100]
segment_cancelations1000 = segment_cancelations[segment_cancelations["TotalBookings"] > 1000]


caption1 = "Pct Cancelations per Segment Type - Bookings > 100" 
caption2 = "Pct Cancelations per Segment Type - Bookings > 1000" 


ViewCancelations_100 = BeautifiedFrame(segment_cancelations100, callback=True, subset=["market_segment", "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCancelations_1000 = BeautifiedFrame(segment_cancelations1000, callback=True,  subset=["market_segment", "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)

display_side_by_side(ViewCancelations_1000, ViewCancelations_100, is_styler=(True, True))


> ### EDA - Distribution Channel vs Cancelations

In [ ]:
distrib_cancelations = pct_aggregator(df, "is_canceled",["distribution_channel"], 1)

distrib_cancelations100 = distrib_cancelations[distrib_cancelations["TotalBookings"] > 100]
distrib_cancelations1000 = distrib_cancelations[distrib_cancelations["TotalBookings"] > 1000]

caption1 = "Pct Cancelations per Distribution Channel - Bookings > 100" 
caption2 = "Pct Cancelations per Distribution Channel - Bookings > 1000" 

ViewCancelations_100 = BeautifiedFrame(distrib_cancelations100, callback=True, subset=["distribution_channel", "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCancelations_1000 = BeautifiedFrame(distrib_cancelations1000, callback=True,  subset=["distribution_channel", "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)

display_side_by_side(ViewCancelations_100, ViewCancelations_1000, is_styler=(True, True))

> ### EDA - Unique [Market Segment + Distribution Channel] vs Cancelations

In [ ]:
campaign_cancelations = pct_multiple_aggregator(df, "is_canceled", 
                                                ["market_segment", "distribution_channel"],
                                                1, 
                                                ascending_order=True)

## Selectig only those campaigns with a significant number of bookings. 
campaign_cancelations100 = campaign_cancelations[campaign_cancelations["TotalBookings"] > 100]
campaign_cancelations1000= campaign_cancelations[campaign_cancelations["TotalBookings"] > 1000]

## Setting the caption title
caption1 = "Campaign Figures - Cancelations with TotalBookings > 100"
caption2 = "Campaign Figures - Cancelations with TotalBookings > 1000"

## Beautifying the Frame. 
ViewCampaigns100 = BeautifiedFrame(campaign_cancelations100, callback=True, subset=["market_segment",
                                                                              "distribution_channel",
                                                                              "TotalBookings"]).set_caption(caption1).set_table_styles(style_props)
ViewCampaigns1000 = BeautifiedFrame(campaign_cancelations1000, callback=True, subset=["market_segment",
                                                                              "distribution_channel",
                                                                              "TotalBookings"]).set_caption(caption2).set_table_styles(style_props)



display_side_by_side(ViewCampaigns100, ViewCampaigns1000, is_styler=(True, True))

> ### EDA - Hotels vs Cancelations

In [ ]:
hotel1_name = "City Hotel"
hotel2_name = "Resort Hotel"

hotel1 = df[df["hotel"] == hotel1_name]
hotel2 = df[df["hotel"] == hotel2_name]

groupby_columns = ["market_segment", "distribution_channel"]

groupby_columns = ["hotel"]  + groupby_columns

hotel1 = hotel1.groupby(groupby_columns)["is_canceled"].count().reset_index().rename(columns={"is_canceled": "TotalBookings"})
hotel2 = hotel2.groupby(groupby_columns)["is_canceled"].count().reset_index().rename(columns={"is_canceled": "TotalBookings"})

In [ ]:

condition1 = (df["is_canceled"] == 1) & (df["hotel"] == hotel1_name)
condition2 = (df["is_canceled"] == 1) & (df["hotel"] == hotel2_name)

hotels1_canceled = df.where(condition1).groupby(groupby_columns)["is_canceled"].count().reset_index().rename(columns={"is_canceled": "TotCancelations"})
hotels2_canceled = df.where(condition2).groupby(groupby_columns)["is_canceled"].count().reset_index().rename(columns={"is_canceled": "TotCancelations"})

hotel1 = pd.merge(hotel1, hotels1_canceled, on=groupby_columns, suffixes=(False, False))
hotel2 = pd.merge(hotel2, hotels2_canceled, on=groupby_columns, suffixes=(False, False))

hotel1["PctCancelations"] = hotel1["TotCancelations"]/hotel1["TotalBookings"]*100
hotel2["PctCancelations"] = hotel2["TotCancelations"]/hotel2["TotalBookings"]*100

hotel1 = hotel1[["hotel", "market_segment", "distribution_channel", "PctCancelations", "TotCancelations", "TotalBookings"]]
hotel2 = hotel2[["hotel", "market_segment", "distribution_channel", "PctCancelations", "TotCancelations", "TotalBookings"]]

hotel1.sort_values(by="PctCancelations", ascending=True, inplace=True)
hotel2.sort_values(by="PctCancelations", ascending=True, inplace=True)

hotel1["PctCancelations"] = hotel1["PctCancelations"].round(2).astype(str) + "  %"
hotel2["PctCancelations"] = hotel2["PctCancelations"].round(2).astype(str) + "  %"


In [ ]:
hotel1_cancelations100 = hotel1[hotel1["TotalBookings"] > 100]
hotel1_cancelations1000 = hotel1[hotel1["TotalBookings"] > 1000]

hotel2_cancelations100 = hotel2[hotel2["TotalBookings"] > 100]
hotel2_cancelations1000 = hotel2[hotel2["TotalBookings"] > 1000]

## Setting the caption title
caption11 = "Campaign Figures ({}) - Cancelations with TotalBookings > 100".format(hotel1_name)
caption12 = "Campaign Figures ({}) - Cancelations with TotalBookings > 1000".format(hotel1_name)
caption21 = "Campaign Figures ({}) - Cancelations with TotalBookings > 100".format(hotel2_name)
caption22 = "Campaign Figures ({}) - Cancelations with TotalBookings > 1000".format(hotel2_name)


## Beautifying the Frame. 

ViewCampaignsHotel1_100 = BeautifiedFrame(hotel1_cancelations100 , callback=True, subset=["market_segment",
                                                                              "distribution_channel",
                                                                              "TotalBookings"]).set_caption(caption11).set_table_styles(style_props)

ViewCampaignsHotel1_1000 = BeautifiedFrame(hotel1_cancelations1000 , callback=True, subset=["market_segment",
                                                                              "distribution_channel",
                                                                              "TotalBookings"]).set_caption(caption12).set_table_styles(style_props)

ViewCampaignsHotel2_100 =BeautifiedFrame(hotel2_cancelations100 , callback=True, subset=["market_segment",
                                                                              "distribution_channel",
                                                                              "TotalBookings"]).set_caption(caption21).set_table_styles(style_props)

ViewCampaignsHotel2_1000 = BeautifiedFrame(hotel2_cancelations100 , callback=True, subset=["market_segment",
                                                                              "distribution_channel",
                                                                              "TotalBookings"]).set_caption(caption22).set_table_styles(style_props)

display_side_by_side(ViewCampaignsHotel1_100, ViewCampaignsHotel1_1000, 
                     ViewCampaignsHotel2_100, ViewCampaignsHotel2_1000,
                     is_styler=(True,True, True, True))

> ### EDA - Other Questions(Ongoing)

*  #### When do operation changes occur?.
*  #### There is any connection between days_in_waiting_list and the rest of the variables?.
*  #### Avg Time between reservation date an arrival?.
*  #### What variables show linear relationship with booking_changes?.


In [ ]:
df[ dfDiscrete.columns.to_list() + dfContinuous.columns.to_list()].corr()["booking_changes"].abs().sort_values(ascending=False)[1:]


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_anova = dfNominal.nunique()
anova_columns = df_anova[df_anova < 10].index.to_list()

df_encoded = df[["booking_changes"] + anova_columns]
df_encoded[anova_columns] = df_encoded[anova_columns].apply(lambda col: le.fit_transform(col))
df_encoded.head(10)

for column in anova_columns:
    print("###########################################################")
    print("######### One-Way Anova for column {} ##########".format(column))
    print("###########################################################")

    
    mod = ols("{}  ~ booking_changes".format(column), data=df_encoded).fit()
    aov_table = sm.stats.anova_lm(mod, typ=2)
    esq_sm = aov_table['sum_sq'][0]/(aov_table['sum_sq'][0] +  aov_table['sum_sq'][1])
    aov_table['EtaSq'] = [esq_sm, 'NaN']
    print(aov_table)


 > ### EDA - Booking Changes vs Customer Type

* #### What can we say about booking_changes?. Lets look at variation per group for all variables from the group of Nominal Variables. In other words :

   *  Do number of booking changes show a relationship with customer type?.


In [ ]:
BookingChangesCustomer = df.groupby(["customer_type"])["booking_changes"].sum().reset_index()
n_changes = df["booking_changes"].sum()
BookingChangesCustomer["Total_Booking_Changes"] = [n_changes]*BookingChangesCustomer.shape[0]
BookingChangesCustomer["PctChanges"] = (BookingChangesCustomer["booking_changes"]/n_changes)*100
BookingChangesCustomer.sort_values(by="PctChanges", ascending=False, inplace=True)
BookingChangesCustomer["PctChanges"] = BookingChangesCustomer["PctChanges"].round(2).astype(str) + " %"
BookingChangesCustomer.style.hide_index().set_properties(**{"text-align":"left"})


 > ### EDA - Booking Changes vs Deposit Type

   *  Do number of booking changes show a relationship with ****deposit type?****.


In [ ]:
BookingChangesDeposit = df.groupby(["deposit_type"])["booking_changes"].sum().reset_index()

n_changes = df["booking_changes"].sum()
BookingChangesDeposit["Total_Booking_Changes"] = [n_changes]*BookingChangesDeposit.shape[0]

BookingChangesDeposit["PctChanges"] = (BookingChangesDeposit["booking_changes"]/n_changes)*100
BookingChangesDeposit.sort_values(by="PctChanges", ascending=False, inplace=True)
BookingChangesDeposit["PctChanges"] = BookingChangesDeposit["PctChanges"].round(2).astype(str) + " %"
BookingChangesDeposit.style.hide_index().set_properties(**{"text-align":"left"})

 > ### EDA - Booking Changes - Time Series (Month/Day)

 * #### Do number of booking changes show a trend when looking at ****time intervals?****

In [ ]:

months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October",
         "November", "December"]

BookingChanges = df.groupby(["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"])["booking_changes"].sum().reset_index()


BookingChanges["arrival_date_month"] = pd.Categorical(BookingChanges["arrival_date_month"], 
                                                            categories = months,
                                                            ordered=True)


columns = ["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"]
AuxDf = BookingChanges.rename(columns={"arrival_date_year": "year",
                                        "arrival_date_month": "month", 
                                        "arrival_date_day_of_month": "day"})

AuxDf["month"] = AuxDf["month"].apply(lambda x : strptime(x, '%B').tm_mon ) 
BookingChanges["Arrival"] = pd.to_datetime(AuxDf[["year", "month", "day"]], format="%Y-%m-%d", errors="coerce")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

time_selection = BookingChanges.where(BookingChanges["arrival_date_year"] == 2015).set_index("Arrival")
time_selection["booking_changes"].plot(linewidth=2)

plt.title("Booking Changes 2015 - Month vs Total Changes", fontsize=25)

In [ ]:
time_selection = BookingChanges.where(BookingChanges["arrival_date_year"] == 2016).set_index("Arrival")
time_selection["booking_changes"].plot(linewidth=2)
plt.title("Booking Changes 2016 - Month vs Total Changes", fontsize=25)


In [ ]:
time_selection = BookingChanges.where(BookingChanges["arrival_date_year"] == 2017).set_index("Arrival")
time_selection["booking_changes"].plot(linewidth=2)
plt.title("Booking Changes 2017 - Month vs Total Changes", fontsize=25)


In [ ]:

BookingChangesByDay = df.groupby(["arrival_date_day_of_month"])["booking_changes"].sum().reset_index()
BookingChangesByDay["booking_changes"].plot(linewidth=2, color="orange")
plt.title("Booking Changes 2015-2017 Daily Totals", fontsize=25)
plt.xticks(list(range(1,31))[::2])

> ### Building a Predictive Model

* Initial set of features to add to our model :

In [ ]:

features = corr_canceled[corr_canceled.columns.values[0]].values
features

In [ ]:
models = ["Random_Forest_Classification",
          "Decission_Tree_Classification",
          "Logistic_Regression", 
          "Support Vector Machine", 
          "Gradient Boosting Tree",
          "Naive Bayes"]

print("Building a Predictive Model : There will be any cancelation?.\n" +
      "We attempt to build a robust predictive model based off of these methods: ")
print("Suggested Models :\n")
print("\n".join( f"{i}. {model}"  for i,model in enumerate(models)) )
print("\n\nSuggested Features - Independent Variables:\n")


features = corr_canceled[corr_canceled.columns.values[0]].to_list()

## As seen in another Kernel booking_changes and days_in_waiting_list may incur in data leakage, that is, the absence of these values while
# while predicting the class of new instance, the values of these two variable might not be there. 
features.remove("booking_changes")
features.remove("days_in_waiting_list")
print("Nummerical Variables : ")
print("\n".join( f"\t> {model}"  for  model in features) )


> ### Random Forest & Decission Tree Classification & Logistic Regression + More.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
def summary_classification(clf):
    print("Number of iterations: ", clf.n_iter_)
    import operator
    coef_weights = { variable:weight for variable, weight in zip(independent_df.columns.values, clf.coef_[0])}
    sorted_coef = dict(sorted(coef_weights.items(), key=operator.itemgetter(1), reverse=False))
    df_coef = pd.DataFrame.from_dict(data=sorted_coef, orient="index")
    print("Intercept: ", clf.intercept_)
    print("Classes: ", clf.classes_ )
    df_coef = df_coef.rename(columns={0: "Weights"})
    return df_coef
    

In [ ]:
n_samples = df.shape[0]

from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier


X = df[features].values
y = df["is_canceled"]

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=5)
print("Decission Tree Classifier", scores.mean())


clf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print("Random Forest Classifier: ", scores.mean())


clf = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print("Extra Trees Classifiesr: " , scores.mean())


#clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.01, random_state=0).fit(X_train, y_train)
#print("Gradient Boosting Tree Score \n with learning rate={} and n_estimators={} : ".format(0.01, 100), clf.score(X_test, y_test))

################### Logistic Regression ##################

from sklearn.linear_model import LogisticRegression


from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, class_weight="balanced", solver="sag").fit(X,y)

clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
clf.score(X, y)



In [ ]:
df_coef = summary_classification(clf)
ViewCoefficients = BeautifiedFrame(df_coef, callback=True, hide_index=False).set_caption("Logistic Regression Coeff. Importance")
ViewCoefficients = ViewCoefficients.set_table_styles(style_props)
ViewCoefficients

* #### Improving the Model with new variables (Feature Selection based on the EDA) we encode the groups of the independent variables that show a higher similarity in variance with cancelations, that is a lowest percentage of cancelations. 


In [ ]:

encoded_columns = ["deposit_type", "customer_type", "market_segment", 
                  "distribution_channel", "country", "reserved_room_type", "meal"]

map_encoding = {"deposit_type" : "No Deposit", 
                "customer_type": "Transient",
                "market_segment": "Direct", 
                "distribution_channel": "Direct",
                "country" : "PRT",
                "reserved_room_type" : "A",
                "assigned_room_type" : "D",
                "meal": "FB"} 

additional_encodings =  {"assigned_room_type" : "A", "reserved_room_type" : "D"} 
        
all_dummies = [ pd.get_dummies(df[K])[V] for K,V in map_encoding.items() ] 
all_dummies += [ pd.get_dummies(df[K])[V] for K,V in additional_encodings.items() ]

encoded_features = pd.concat(all_dummies, axis=1)

### Add in the horizontal axis the columns of all the independent variables that will be part of our next model. 
dic_columns = ["hotel", "is_repeated_guest"]
dummies_hotel = pd.get_dummies(df["hotel"])
independent_df = pd.concat([df[features], encoded_features, dummies_hotel, df["is_repeated_guest"]], axis=1)
 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

X = independent_df.values
y = df["is_canceled"]

##### KFolds #####
kfolds = 4
split = KFold(n_splits=kfolds, shuffle=True, random_state=42)


### Decission Tree Classifier
clf = DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0)

scores = cross_val_score(clf, X, y, cv=5)
print("Decission Tree Classifier : ", scores.mean())


### Decission Tree Classifier hyperparameters tuned. 

clf = DecisionTreeClassifier(random_state = 0, max_features="auto", min_impurity_decrease=1e-6)
cv_results = cross_val_score(clf, X, y, cv=split, scoring="accuracy", n_jobs=-1)
print("Decission Tree Classifier Tuned [Score]: {}".format(cv_results.mean()))


### Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print("Random Forest Classifier: ", scores.mean())

### Random Forest Classifier Tuned
clf = RandomForestClassifier(n_estimators=120, max_depth=None, min_samples_split=2, random_state=42)
scores = cross_val_score(clf, X, y, cv=split, scoring="accuracy", n_jobs=-1)
print("Random Forest Classifier Tuned: ", scores.mean())

### Extra Trees Classifier
clf = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X, y, cv=5)
print("Extra Trees Classifiesr: " , scores.mean())


### Gradient Boosting Classifier
#clf = GradientBoostingClassifier(n_estimators=120, learning_rate=0.01, random_state=0).fit(X_train, y_train)
#print("Gradient Boosting Tree Score : {} \n with learning rate={} and n_estimators={} : ".format(clf.score(X_test, y_test),
                                                                                                 #0.01, 100))


### Logistic Regression Classifier
clf = LogisticRegression(random_state=42, n_jobs=-1).fit(X,y)

kfolds = 4
split = KFold(n_splits=kfolds, random_state=42, shuffle=True)

scores = cross_val_score(clf, X, y, cv=split, scoring="accuracy", n_jobs=-1)
print("Logistic Regression accuracy:" , scores.mean())





> ## CONCLUSIONS AND TAKEAWAYS

* Random Forest Classification with a number of estimators around 110~130 shows good accuracy numbers 86%. 

* Artificial Neural Network will surely notch up the prediction by learning other relationships than we are not able to spot or learn using machine learning or statistics. 

* Focusing on designing useful pipelines that do the always tedious task of preparing, cleaning data is key. 

* By adding hand picked features we can incur into high bias, that is that for new examples, that is the model may fail to generalize for new examples. 

* By structuring Data Science Projects together with the honing of knowledge in pandas methods, we not only shorten our times in Wrangling, Cleaning and Processing Data, we are able to see as well the growing need of the utilization of higher levels of abstraction. The more structured and neatly planned the processes are the easier to spot anomalies and patterns in later stages throughout the work, to facilitate an orderly univariate, bivariate or multivariate analysis. 


